In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu b

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-16 04:48:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-07-16 04:48:21 (6.43 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 177 µs (started: 2022-07-16 04:48:43 +00:00)


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Customer-Segmentation").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

time: 7.87 s (started: 2022-07-16 04:49:49 +00:00)


In [11]:

jdbc_url="jdbc:postgresql://bootcamp15.cx9czezoxozl.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"CS_bootcamp", 
          "password": "xxxxxxxx", ## password removed
          "driver":"org.postgresql.Driver"}

time: 2.07 ms (started: 2022-07-16 04:59:28 +00:00)


In [12]:
df =  spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "public.customer") \
    .option("user", "xxxxxxxx") \ ## User name removed
    .option("password", "xxxxxxxxxxxx") \ ## password removed
    .option("driver", "org.postgresql.Driver") \
    .load()

#spark.read.jdbc(url=jdbc_url, table='customer',properties=config)

time: 2.62 s (started: 2022-07-16 04:59:31 +00:00)


In [19]:
df.show()

+----+----------+----------+-----------------+------+-------+--------+-------------+--------------+------------+-----------+
|  id|year_birth| education|martial_status_id|income|kidhome|teenhome|year_acquired|month_acquired|day_acquired|dt_customer|
+----+----------+----------+-----------------+------+-------+--------+-------------+--------------+------------+-----------+
|5524|      1957|Graduation|                1| 58138|      0|       0|         2012|             9|           4| 2012-09-04|
|2174|      1954|Graduation|                1| 46344|      1|       1|         2014|             3|           8| 2014-03-08|
|4141|      1965|Graduation|                3| 71613|      0|       0|         2013|             8|          21| 2013-08-21|
|6182|      1984|Graduation|                3| 26646|      1|       0|         2014|             2|          10| 2014-02-10|
|5324|      1981|       PhD|                2| 58293|      1|       0|         2014|             1|          19| 2014-01-19|


In [17]:
dfp = df.toPandas()

time: 4.4 s (started: 2022-07-16 05:00:42 +00:00)


In [18]:
dfp.head()

,id,year_birth,education,martial_status_id,income,kidhome,teenhome,year_acquired,month_acquired,day_acquired,dt_customer
0,5524,1957,Graduation,1,58138.0,0,0,2012,9,4,2012-09-04
1,2174,1954,Graduation,1,46344.0,1,1,2014,3,8,2014-03-08
2,4141,1965,Graduation,3,71613.0,0,0,2013,8,21,2013-08-21
3,6182,1984,Graduation,3,26646.0,1,0,2014,2,10,2014-02-10
4,5324,1981,PhD,2,58293.0,1,0,2014,1,19,2014-01-19


time: 22.9 ms (started: 2022-07-16 05:00:48 +00:00)
